### Notebook written and maintained by Alex Jones (alexander.g.jones.23@dartmouth.edu, alexjones1925@gmail.com)

### May be used for replicating Tatoeba dataset results from paper "Majority Voting with Bidirectional Pre-translation for Bitext Retrieval" (Alex Jones and Derry Tanti Wijaya)

### Code for margin-based similarity search based on LASER implementation by Facebook AI: https://github.com/facebookresearch/LASER/blob/master/source/mine_bitexts.py

### See https://github.com/facebookresearch/LASER for copyright and licensing specifications

In [10]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import time # for timing processes or providing time estimates
import faiss # for vector similarity search
from sentence_transformers import SentenceTransformer, util # sentence embedding model
from googleapiclient.discovery import build # Google Cloud Translation
import getpass # Google Cloud authentication

In [2]:
# Checks for GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
# we use LaBSE for this project, see Sentence Transformers documentation
# for other options
sentence_model = SentenceTransformer('LaBSE')
sentence_model.cuda() # puts model on GPU

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [4]:
# wrapper for embedding function
def sentence_embed(sentences:list):
    return sentence_model.encode(sentences)

In [5]:
GPU = faiss.StandardGpuResources() # enables GPU for similarity search

In [1]:
'''

Params
******
src_emb: array of size number_of_source_sentences X embedding_dimension
tgt_emb: array of size number_of_target_sentences X embedding_dimension
k: number of neighbors to return
batch_size: batch size

Returns
*******
cos_sims: cosine similarity scores for each of k nearest neighbors for each source sentence
inds: target indices of k nearest neighbors for each source sentence

Modeled off of LASER source code: https://github.com/facebookresearch/LASER/blob/master/source/mine_bitexts.py

'''

def knnSearch(src_emb, tgt_emb, k=1, batch_size=1):
    emb_dim = src_emb.shape[1] # Embedding dimension
    num_src_sents = src_emb.shape[0]
    num_tgt_sents = tgt_emb.shape[0]
    cos_sims = np.zeros((num_src_sents, k), dtype=np.float32)
    inds = np.zeros((num_src_sents, k), dtype=np.int64)
    for s_min in range(0, num_src_sents, batch_size):
        s_max = min(s_min + batch_size, num_src_sents)
        src_sims = []
        src_inds = []
        for t_min in range(0, num_tgt_sents, batch_size):
            t_max = min(t_min + batch_size, num_tgt_sents)
            idx = faiss.IndexFlatIP(emb_dim)
            idx = faiss.index_cpu_to_gpu(GPU, 0, idx)
            idx.add(tgt_emb[t_min : t_max])
            src_sim, src_ind = idx.search(src_emb[s_min : s_max], min(k, t_max-t_min))
            src_sims.append(src_sim)
            src_inds.append(src_ind + t_min)
            del idx
        src_sims = np.concatenate(src_sims, axis=1)
        src_inds = np.concatenate(src_inds, axis=1)
        sorted_inds = np.argsort(-src_sims, axis=1)
        for i in range(s_min, s_max):
            for j in range(k):
                cos_sims[i, j] = src_sims[i-s_min, sorted_inds[i-s_min, j]]
                inds[i, j] = src_inds[i-s_min, sorted_inds[i-s_min, j]]
    return cos_sims, inds

In [7]:
# Retrieves k-nearest neighbor indices and similarity means for margin scoring
# If forward: finds neearest neighbors and indices for all source sentences
# If backward: finds nearest neighbors and indices for all target sentences
# In the approach implemented in our paper, we perform both forward and backward search

def directedMeansAndInds(src_emb, tgt_emb, forward=False, backward=False, k=1, batch_size=1):
    assert forward != backward, "Please choose either forward or backward"
    if forward:
        cos_sims, inds = knnSearch(src_emb, tgt_emb, min(tgt_emb.shape[0], k), batch_size)
        return cos_sims.mean(axis=1), inds
    elif backward:
        cos_sims, inds = knnSearch(tgt_emb, src_emb, min(src_emb.shape[0], k), batch_size)
        return cos_sims.mean(axis=1), inds

In [8]:
'''

Params
******
pred_tuples: predicted sentence pairs
gold_tuples: ground-truth sentence pairs

Returns
*******
Unweighted F1, precision, recall

'''

def computeF1(pred_tuples, gold_tuples):
    tp = 0 # true positives
    fp = 0 # false positives
    prec = 0
    rec = 0
    f1 = 0
    epsilon = 1e-8 # To prevent division by zero
    for pair in pred_tuples:
        if pair in gold_tuples:
            tp += 1
        else:
            fp += 1 
    prec = tp / (len(pred_tuples) + epsilon)
    rec = tp / len(gold_tuples)
    f1 = 2*prec*rec / (prec+rec+epsilon)
    return f1, prec, rec

In [2]:
'''

Params
******
src_embs: array of size number_of_source_sentences X embedding_dimension
tgt_embs: array of size number_of_source_sentences X embedding_dimension
batch_size: batch size
num_neighbors: number of neighbors

Returns
*******
concat_pairs: list of mined sentence pairs
margin_scores: list of scores corresponding to mined pairs

'''


def mineSentencePairs(src_embs, tgt_embs, batch_size=100, num_neighbors=4):

    # Retrieve means and indices in forward direction . . .
    fwd_means, fwd_inds = directedMeansAndInds(src_embs, tgt_embs, forward=True, k=num_neighbors, batch_size=batch_size)
    # . . . and in backward direction
    bwd_means, bwd_inds = directedMeansAndInds(src_embs, tgt_embs, backward=True, k=num_neighbors, batch_size=batch_size)

    fwd_margin_scores = np.zeros(fwd_inds.shape)
    for i in range(fwd_inds.shape[0]):
        for j in range(fwd_inds.shape[1]):
            tgt_ind = fwd_inds[i,j]
            # Compute ratio margin score between each source sentence and each of its k-nearest neighbors
            margin_score = (src_embs[i].dot(tgt_embs[tgt_ind])) / np.average((fwd_means[i], bwd_means[tgt_ind]))
            # Store the result
            fwd_margin_scores[i,j] = margin_score
    
    # We will store the source index, target index, and margin score for the best
    # pairs found using forward search
    best = np.zeros((fwd_inds.shape[0], 3))
    # Take pair that maximizes margin score for each source sentence
    best_inds = fwd_inds[np.arange(src_embs.shape[0]), fwd_margin_scores.argmax(axis=1)]
    for i in range(fwd_inds.shape[0]):
        best_score, ind = (np.max(fwd_margin_scores[i]), np.argmax(fwd_margin_scores[i]))
        best[i] = ((i+1, best_inds[i]+1, best_score)) # Assumption is that GROUND TRUTH VALUES ARE 1-INDEXED!!!

    # Repeat process in backward direction (finding matches in source text for target sentences)
    bwd_margin_scores = np.zeros(bwd_inds.shape)
    for i in range(bwd_inds.shape[0]):
        for j in range(bwd_inds.shape[1]):
            tgt_ind = bwd_inds[i,j]
            margin_score = (tgt_embs[i].dot(src_embs[tgt_ind])) / np.average((bwd_means[i], fwd_means[tgt_ind]))
            bwd_margin_scores[i,j] = margin_score
            
    bwd_best = np.zeros((bwd_inds.shape[0], 3))
    best_inds = bwd_inds[np.arange(tgt_embs.shape[0]), bwd_margin_scores.argmax(axis=1)]
    for i in range(bwd_inds.shape[0]):
        best_score, ind = (np.max(bwd_margin_scores[i]), np.argmax(bwd_margin_scores[i]))
        bwd_best[i] = ((best_inds[i]+1, i+1, best_score))
    
    # Best triples (src_idx, tgt_idx, margin_score) from forward/backward searches
    fwd_best = [tuple(best[i]) for i in range(best.shape[0])]
    bwd_best = [tuple(bwd_best[i]) for i in range(bwd_best.shape[0])]

    pairs_and_scores = []
    # Take INTERSECTION of forward and backward searches
    pairs_and_scores = list(set(fwd_best) & set(bwd_best))

    pairs_and_scores = list(dict.fromkeys(pairs_and_scores))
    concat_pairs = [(triplet[0], triplet[1]) for triplet in pairs_and_scores] # Store indices only
    concat_pairs_int = []
    for tup in concat_pairs:
        concat_pairs_int.append((int(tup[0]), int(tup[1]))) # Ground-truth indices are ints, so change type
    concat_pairs = concat_pairs_int

    margin_scores = [triplet[2] for triplet in pairs_and_scores] # Store scores only
                                    
    return concat_pairs, margin_scores

In [ ]:
# You'll need a Google Cloud account and an API key to use the Cloud Translate API
APIKEY = getpass.getpass()

# Instantiate the translation API
service = build('translate', 'v2', developerKey=APIKEY)

# Function for batch translating sentences between any two supproted languages
# Note that the src_code/tgt_code are TWO-letter ISO codes (see below for Tatoeba list)
def translateGoogle(sentences: list, src_code: str, tgt_code: str):
    transl_doc = (((service.translations().list(source=src_code, target=tgt_code, q=sentences, format='text').execute())))#['translations'])[0])['translatedText']
    return [sent['translatedText'] for sent in transl_doc['translations']]

In [3]:
'''

Params
******
lang_id_3: three-letter ISO code used to load Tatoeba file
lang_id_2: two-letter Iso code used as input for Cloud Translation
directory: the directory where your Tatoeba files are stored
verbose: boolean indicating whether you want results printed in real time

Returns
*******
all_results: list containing unweighted F1, precision, and recall for each
of the given methods, for each of the Tatoeba language pairs

'''

def retrieveTatoebaComparisons(lang_id_3, lang_id_2, directory, verbose=True):
    
    start = time.time()
    
    # We'll store the results here and return them at the end
    all_results = []
    
    print("Reading, translating, embedding . . .")
    with open('{}/tatoeba.{}-eng.eng'.format(directory, lang_id_3), 'r') as f:
        en_sents = f.read().splitlines()
    with open('{}/tatoeba.{}-eng.{}'.format(directory, lang_id_3, lang_id_3), 'r') as f:
        xx_sents = f.read().splitlines()
        
    # Translate en sentences to xx
    en_to_xx_sents = [translateGoogle([en_sent], 'en', lang_id_2)[0] for en_sent in en_sents]
    # Translate xx sentences to en
    xx_to_en_sents = [translateGoogle([xx_sent], lang_id_2, 'en')[0] for xx_sent in xx_sents]
    
    # Create embeddings of en sentences, xx sentences, 
    # translated en sentences, and translated xx sentences
    en_emb = embed(en_sents)
    xx_emb = embed(xx_sents)
    en_to_xx_emb = embed(en_to_xx_sents)
    xx_to_en_emb = embed(xx_to_en_sents)
    
    # Tatoeba sentences are already paired and sorted, so the ground-truth labels
    # are simply {(1, 1), (2, 2), . . . , (n, n)}, where n is the number of sentences
    ground_truth_pairs = [(i+1,i+1) for i in range(len(en_sents))]
    
    print("Mining and scoring . . .")
    # METHOD 1: Margin scoring, no threshold
    sent_pairs_og, marg_scores_og = mineSentencePairs(en_emb, xx_emb)
    F1, P, R = computeF1(sent_pairs_og, ground_truth_pairs)
    if verbose:
        print("F1, P, R for {} with no threshold: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 2: Margin scoring, threshold = 1.06
    above_threshold = []
    THRESHOLD = 1.06
    for i in range(len(sent_pairs_og)):
        if marg_scores_og[i] > THRESHOLD:
            above_threshold.append(sent_pairs_og[i])
    F1, P, R = computeF1(above_threshold, ground_truth_pairs)
    if verbose:
        print("F1, P, R for {} with threshold = 1.06: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 3: Margin scoring, threshold = 1.20
    above_threshold = []    
    THRESHOLD = 1.20
    for i in range(len(sent_pairs_og)):
        if marg_scores_og[i] > THRESHOLD:
            above_threshold.append(sent_pairs_og[i])
    F1, P, R = computeF1(above_threshold, ground_truth_pairs)
    if verbose:
        print("F1, P, R for {} with threshold = 1.20: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 4: Margin scoring, translation en->xx, no threshold
    sent_pairs_en_xx, marg_scores_en_xx = mineSentencePairs(en_to_xx_emb, xx_emb)
    F1, P, R = computeF1(sent_pairs_en_xx, ground_truth_pairs)
    if verbose:
        print("F1, P, R for {} using translation en-xx: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 5: Margin scoring, translation xx->en, no threshold
    sent_pairs_xx_en, marg_scores_xx_en = mineSentencePairs(en_emb, xx_to_en_emb)
    F1, P, R = computeF1(sent_pairs_xx_en, ground_truth_pairs)
    if verbose:
        print("F1, P, R for {} using translation xx_en: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 6: Intersection of methods 1, 4, and 5
    sent_pairs_triple_intersection = list(set(sent_pairs_og)&set(sent_pairs_en_xx)&set(sent_pairs_xx_en))
    F1, P, R = computeF1(sent_pairs_triple_intersection, ground_truth_pairs)
    if verbose:
        print("F1, P, R for {} using triple intersection: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 7: Majority vote intersection of methods 1, 4, and 5
    int1 = set(sent_pairs_og)&set(sent_pairs_en_xx)
    int2 = set(sent_pairs_en_xx)&set(sent_pairs_xx_en)
    int3 = set(sent_pairs_og)&set(sent_pairs_xx_en)
    sent_pairs_majority_vote = list(set(int1|int2|int3))
    F1, P, R = computeF1(sent_pairs_majority_vote, ground_truth_pairs)
    if verbose:
        print("F1, P, R for {} using majority vote intersection: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    end = time.time()
    if verbose:
        print("Time taken: {:.2f} seconds".format(end-start))
    
    return all_results

In [13]:
# THREE-LETTER ISO codes: used for loading Tatoeba files
lang_codes_3c = ['afr', 'amh', 'ang', 'arq', 'arz', 'ast',
                 'awa', 'aze', 'bel', 'ben', 'ber', 'bos',
                 'bre', 'cbk', 'ceb', 'cha', 'cor', 'csb', 
                 'cym', 'dsb', 'dtp', 'epo', 'eus', 'fao', 
                 'fry', 'gla', 'gle', 'gsw', 'hsb', 'ido', 
                 'ile', 'ina', 'isl', 'jav', 'kab', 'kaz', 
                 'khm', 'kur', 'kzj', 'lat', 'lfn', 'mal', 
                 'maz', 'mhr', 'nds', 'nno', 'nov', 'oci', 
                 'orv', 'pam', 'pms', 'swg', 'swh', 'tam', 
                 'tat', 'tel', 'tgl', 'tuk', 'tzl', 'uig', 
                 'uzb', 'war', 'wuu', 'xho', 'yid'] 

# TWO-LETTER ISO codes: used for Cloud Translation
# "False" is given when the language isn't supported by Cloud Translation
lang_codes_2c = ['af', 'am', False, False, False, False, 
                 False, 'az', 'be', 'bn', False, 'bs',
                 False, False, 'ceb', False, 'co', False,
                 'cy', 'pl', False, 'eo', 'eu', False, 
                 'fy', 'gd', 'ga', 'pl', 'eo', 'la',
                 'la', 'is', 'jv', False, 'kk', 'km',
                 'ku', False, 'lv', False, 'mt', False,
                 False, 'de', 'no', False, False, False,
                 False, False, False, 'sw', 'ta', 'tt',
                 'te', 'tl', 'tk', False, 'ug', 'uz',
                 False, False, 'xh', 'yi']

In [ ]:
# Retrieving results for Tatoeba language pairs with Cloud Translation support

start = time.time()
results_by_lang = []
for l3,l2 in zip(lang_codes_3c, lang_codes_2c):
    if l2:
        res = retrieveTatoebaComparisons(l3, l2)
        results_by_lang.append(res)
    else:
        results_by_lang.append((l3, "NULL")) # If language lacks translation support, flag as NULL
end = time.time()
print("Total time taken: {:.2f} seconds".format(end-start))

In [12]:
# Languages without Cloud Translation support
missing_langs = ['ang','arq', 'arz', 'ast', 'awa', 'ber',
                 'bre', 'cbk', 'cha', 'csb', 'dtp', 'fao',
                 'jav', 'kzj','lfn', 'mhr', 'nov', 
                 'oci', 'orv', 'pam', 'pms', 'swg', 'tzl',
                 'war', 'wuu']

In [4]:
# Retrieving results for language pairs lacking Cloud Translation support

start = time.time()
missing_results = []
for l3 in missing_langs:
    res = retrieveTatoebaComparisons(l3, '')
    missing_results.append(res)
end = time.time()
print("Total time taken: {:.2f} seconds".format(end-start))

NameError: name 'time' is not defined